In [ ]:
本文档禁止ai修改

## 📊 实验汇总

### ARC-Challenge-test集 测试结果详细记录

### 表格 1：Llama-3.2-3B (source) 使用 ARC-Challenge-train 微调后迁移至 Qwen2.5-1.5B (target)

| 模型                           | 准确率(ACC) | 提升     | 微调/迁移配置                                                 |
| ---------------------------- | -------- | ------ | ------------------------------------------------------- |
| Llama-3.2-3B                 | 0.6838   | -      | -                                                       |
| Llama-3.2-3B + LoRA          | 0.7082   | +2.44% | steps=600, batch=6, lr=1.5e-5                           |
| Qwen2.5-1.5B (baseline)      | 0.7338   | -      | -                                                       |
| Qwen2.5-1.5B + 从llama迁移的LoRA | 0.7372   | +0.34% | target: q, k, v, o, gate, up, down，共396层，max\_sim=0.002 |
| Qwen2.5-1.5B + LoRA          | 0.7457   | +1.19% | steps=600, batch=6, lr=1.5e-5                           |

---

### 表格 2：Qwen2.5-1.5B (source) 使用 ARC-Challenge-train 微调后迁移至 Llama-3.2-3B (target)

| 模型                          | 准确率(ACC) | 提升     | 微调/迁移配置                                                 |
| --------------------------- | -------- | ------ | ------------------------------------------------------- |
| Qwen2.5-1.5B                | 0.7338   | -      | -                                                       |
| Qwen2.5-1.5B + LoRA         | 0.7457   | +1.19% | steps=600, batch=6, lr=1.5e-5                           |
| Llama-3.2-3B (baseline)     | 0.6838   | -      | -                                                       |
| Llama-3.2-3B + 从Qwen迁移的LoRA | 0.7065   | +2.27% | target: q, k, v, o, gate, up, down，共396层，max\_sim=0.002 |
| Llama-3.2-3B + LoRA         | 0.7082   | +2.44% | steps=600, batch=6, lr=1.5e-5                           |

---




In [ ]:
微调lora
有早停 动态学习率 每个epoch后用valid选best model

cd /root/PAW/train_lora && python train_cs_lora_lightning.py --dataset arc-challenge --base_model /root/autodl-tmp/models/Llama-3.2-3B-Instruct --bs 4 --max_steps 600

In [ ]:
迁移

qwen 迁移到 llama
cd /root/PAW/lora_adapter && python scripts/transfer_lora_x.py   --source_lora /root/PAW/train_lora/runs/Qwen_Qwen2.5-1.5B/arc-challenge_lora_20250724_014727/final_model   --source_model /root/autodl-tmp/models/Qwen_Qwen2.5-1.5B   --target_model /root/autodl-tmp/models/Llama-3.2-3B-Instruct   --output /root/autodl-tmp/adapted_lora/qwen_to_llama_lora_x_fixed   --similarity_threshold 0.0001

llama 迁移到 qwen
cd /root/PAW/lora_adapter && python scripts/transfer_lora_x.py   --source_lora /root/PAW/train_lora/runs/Llama-3.2-3B-Instruct/arc-challenge_lora_20250724_140508/final_model   --source_model /root/autodl-tmp/models/Llama-3.2-3B-Instruct   --target_model //root/autodl-tmp/models/Qwen_Qwen2.5-1.5B   --output /root/autodl-tmp/adapted_lora/llama_to_qwen   --similarity_threshold 0.0001

In [ ]:
eval 迁移过的

qwen 迁移到 llama
python eval/lightning_eval.py --models_list /root/autodl-tmp/adapted_lora/qwen_to_llama_lora_x_fixed --dataset arc-challenge --sample_ratio 0.1 --base_model /root/autodl-tmp/models/Llama-3.2-3B-Instruct

llama 迁移到 qwen
python eval/lightning_eval.py --models_list /root/autodl-tmp/adapted_lora/llama_to_qwen --dataset arc-challenge --sample_ratio 0.1 --base_model /root/autodl-tmp/models/Qwen_Qwen2.5-1.5B

In [ ]:
eval base model

qwen
python eval/lightning_eval.py --models_list /root/autodl-tmp/adapted_lora/llama_to_qwen --dataset arc-challenge --sample_ratio 0.1 --base_model /root/autodl-tmp/models/Qwen_Qwen2.5-1.5B

llama

In [ ]:
python pipeline/experiments/run_single_experiment.py --base_model /root/autodl-tmp/models/Qwen_Qwen2.5-1.5B --target_model /root/autodl-tmp/models/Llama-3.2-3B-Instruct --dataset arc-challenge --eval_only